**SQL tasks graduation project Yandex Practicum Data Analyst Plus**

**Target**

explore the database and answer the questions

**Data Description**

**Table `books`**

Contains information about books:

- `book_id` — book identifier;
- `author_id` — author identifier;
- `title` — title of the book;
- `num_pages` - number of pages;
- `publication_date` - publication date of the book;
- `publisher_id` - publisher identifier.

**The `authors` table**

Contains information about the authors:

- `author_id` — author identifier;
- `author` is the name of the author.

**Table `publishers`**

Contains information about publishers:

- `publisher_id` — publisher identifier;
- `publisher` — publisher name;

**Table `ratings`**

Contains data about user ratings of books:

- `rating_id` — rating identifier;
- `book_id` — book identifier;
- `username` - the name of the user who left the rating;
- `rating` - rating of the book.

**Table `reviews`**

Contains data about user reviews of books:

- `review_id` — review identifier;
- `book_id` — book identifier;
- `username` - the name of the user who wrote the review;
- `text` — review text.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp',
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

**1 Count how many books have been published since January 1, 2000**

In [2]:
query = """
SELECT COUNT(books) 
FROM books
WHERE publication_date >= CAST('2000-01-01' as date)
"""
pd.io.sql.read_sql(query, con = engine) 

,count
0,821


**2 For each book, count the number of reviews and the average rating**

In [3]:
query = """
WITH a AS (SELECT title, COUNT(review_id) as cnt
FROM books LEFT JOIN reviews USING(book_id)
GROUP BY title),
b AS (SELECT title, AVG(rating)
FROM books LEFT JOIN ratings USING(book_id)
GROUP BY title)
SELECT * FROM a LEFT JOIN b USING(title)
ORDER BY cnt DESC
"""

pd.io.sql.read_sql(query, con = engine) 

,title,cnt,avg
0,Memoirs of a Geisha,8,4.107143
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


**3 Identify the publisher that has released the most books over 50 pages - this way you will exclude brochures from the analysis**

In [4]:
query = """
WITH
a AS (SELECT publisher_id, COUNT(book_id) FROM books
WHERE num_pages>50
GROUP BY publisher_id
ORDER BY COUNT(book_id) DESC
LIMIT 1)

SELECT DISTINCT publisher
FROM books as b JOIN publishers as p USING(publisher_id)
WHERE b.publisher_id = (SELECT publisher_id FROM a)

"""

pd.io.sql.read_sql(query, con = engine) 

,publisher
0,Penguin Books


**4 Identify the author with the highest average book rating - consider only books with 50 or more ratings**

In [5]:
query = """
WITH 
a AS (SELECT book_id, COUNT(rating) 
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) >50)
SELECT author, AVG(rating) 
FROM books JOIN ratings USING(book_id) JOIN authors USING(author_id)
WHERE book_id IN (SELECT book_id FROM a)
GROUP BY(author)
ORDER BY AVG(rating) DESC
LIMIT 1

"""

pd.io.sql.read_sql(query, con = engine) 

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


**5 Calculate the average number of reviews from users who have given more than 50 ratings**

In [6]:
query = """
WITH 
a AS (SELECT username, COUNT(rating_id)
FROM ratings
GROUP BY username
HAVING COUNT(rating_id)>50),
b as (SELECT username, COUNT(review_id) as cnt
FROM reviews
WHERE username in (SELECT username FROM a)
GROUP BY username)
SELECT AVG(cnt)
FROM b
"""

pd.io.sql.read_sql(query, con = engine) 

,avg
0,24.333333
